In [1]:
import ee
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [ ]:
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              8.473892211914062,
              49.98081240937428
            ],
            [
              8.658599853515625,
              49.98081240937428
            ],
            [
              8.658599853515625,
              50.06066538593667
            ],
            [
              8.473892211914062,
              50.06066538593667
            ],
            [
              8.473892211914062,
              49.98081240937428
            ]
          ]
        ]
      }
    }
  ]
}

In [ ]:
coords = geoJSON 'features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

#поле
28.327897562397894,55.039428955902046,
28.330060162226996,55.038648634224636,
28.330800456547973,55.03798868780123,
28.33114380399439,55.03814025695209,
28.330813815502186,55.038563883624256,
28.33395752670043,55.04182797430781,
28.330871760040484,55.04259937649256,
28.327897562397894,55.039428955902046



#лес
28.36867608564167,55.022582459441814,
28.36874294658985,55.02206521058777,
28.37454865454415,55.022595858779546,
28.374267725132615,55.02293029316314,
28.373870879521395,55.0232602426756,
28.373870974775777,55.02355012908304,
28.373719364548023,55.02405400731252,
28.373902022182207,55.024437455463755,
28.374892055329326,55.02461580420657,
28.374562011980984,55.02507958796338,
28.374419281096788,55.02528022215966,
28.375007949154853,55.02546751596582,
28.37703238535213,55.025886691095124,
28.376885185039708,55.02594464914051,
28.37664442489605,55.0261988339464,
28.37680052237374,55.02664921978632,
28.37665341404765,55.02686764479318,
28.376006759168504,55.02698810213807,
28.37531566170606,55.02697472927314,
28.37506145211032,55.02669827258685,
28.374575496790893,55.02688998306344,
28.374178574655723,55.02687214245029,
28.37390654361895,55.026372757851,
28.37359895534249,55.026216652429945,
28.373665700642125,55.02593572399014,
28.373233325777882,55.025708315786225,
28.372617852409682,55.02558790304464,
28.372577745453444,55.02528920026555,
28.372805243675966,55.02497702445131,
28.37239052438747,55.02469164662594,
28.37136041079541,55.02518662221359,
28.37086545748986,55.024861136968426,
28.36998250357148,55.02419220135768,
28.369108621469234,55.02339401848582,
28.36867608564167,55.0

In [ ]:
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

In [31]:
import pandas as pd
from dateutil.relativedelta import relativedelta
# Any region of the world
polygon = ee.Geometry.Polygon([27.129564922715716,54.55896956167419,
27.14295451011806,54.55896956167419,
27.14295451011806,54.566932014707845,
27.129564922715716,54.566932014707845,
27.129564922715716,54.55896956167419
])
startDate = '2021-04-01'
endDate = '2022-11-30'

modisNDVI = ee.ImageCollection('MODIS/MOD09GA_006_NDVI').select('NDVI').filterDate(startDate, endDate)
# Select the bands for NDVI calculation


start_date = pd.to_datetime(startDate)
end_date = pd.to_datetime(endDate)

# Calculate the number of months between the two dates
num_months = relativedelta(end_date, start_date).months + (relativedelta(end_date, start_date).years * 12)

print(num_months)
    
def custom_fun(n):
    date = ee.Date(startDate).advance(n,'month')
    m = date.get("month")
    y = date.get("year")
    dic = ee.Dictionary({
        'Date':date.format('yyyy-MM')
    })
    
    tempNDVI = (modisNDVI.filter(ee.Filter.calendarRange(y, y, 'year'))
                .filter(ee.Filter.calendarRange(m, m, 'month'))
                .mean()
                .reduceRegion(
                    reducer = ee.Reducer.mean(),
                    geometry = polygon,
                    scale = 250))

    return dic.combine(tempNDVI)

modis_YrMo = ee.List.sequence(0, num_months).map(custom_fun)

dataframe = pd.DataFrame(modis_YrMo.getInfo())

19


In [32]:
dataframe.head(num_months)

,Date,NDVI
0,2021-04,0.174130
1,2021-05,0.205487
2,2021-06,0.518721
3,2021-07,0.520979
4,2021-08,0.207294
5,2021-09,0.235497
6,2021-10,0.301212
7,2021-11,0.129781
8,2021-12,0.006767
9,2022-01,0.048377


In [33]:
dataframe['Date'] = pd.to_datetime(dataframe['Date'])

dataframe = dataframe[(dataframe['Date'].dt.month >= 4) & (dataframe['Date'].dt.month <= 6) | (dataframe['Date'].dt.month >= 9) & (dataframe['Date'].dt.month <= 11)]

march_to_may = (dataframe['Date'].dt.month >= 3) & (dataframe['Date'].dt.month <= 6)
sept_to_nov = (dataframe['Date'].dt.month >= 9) & (dataframe['Date'].dt.month <= 11)

if (dataframe[march_to_may]['NDVI'].min() < 0.16) & (dataframe[sept_to_nov]['NDVI'].min() < 0.16):
    print("Changes detected")
else:
    print("Changes not detected")

Changes detected
